In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/02 07:23:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-06.csv.gz')

In [5]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [8]:
from pyspark.sql import types

In [9]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropoff_datetime', types.TimestampType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True), 
    types.StructField('Affiliated_base_number', types.IntegerType(), True)
])

In [10]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-06.csv.gz')

In [14]:
df = df.repartition(12)

In [15]:
df.write.parquet('fhvhv/2021/06/')

In [46]:
df = spark.read.parquet('fhvhv/2021/06/')

In [17]:
df

DataFrame[dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string, Affiliated_base_number: int]

In [18]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02866|2021-06-20 23:48:47|2021-06-21 00:11:11|         138|         256|      N|                  null|
|              B02835|2021-06-12 08:19:10|2021-06-12 08:42:20|          77|         215|      N|                  null|
|              B02869|2021-06-13 17:19:26|2021-06-13 17:23:57|         152|         166|      N|                  null|
|              B02877|2021-06-22 19:15:39|2021-06-22 19:34:59|         254|         243|      N|                  null|
|              B02510|2021-06-23 21:51:53|2021-06-23 22:09:36|         260|          25|      N|                  null|
|              B02765|2021-06-24 22:58:3

In [23]:
from pyspark.sql import functions as F

In [35]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID' , 'DOLocationID') \
    .where("pickup_date = '2021-06-15'") \
    .count()

452470

In [47]:
df.createOrReplaceTempView('trips')

In [ ]:
spark.sql("""
SELECT MAX(DATEDIFF(hour, pickup_datetime, dropoff_datetime)) FROM trips
""").show()

In [63]:
spark.sql("""
SELECT PULocationID, COUNT(PULocationID) AS frequency
FROM trips
GROUP BY PULocationID
ORDER BY frequency DESC
LIMIT 1;
""").show()


+------------+---------+
|PULocationID|frequency|
+------------+---------+
|          61|   231279|
+------------+---------+

